In [1]:
import tensorflow as tf
import reader
import os

In [2]:
raw_data = reader.ptb_raw_data('simple-examples/data/')
train_data, valid_data, test_data, _ = raw_data 

In [3]:
class SmallConfig(object):
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

In [4]:
config = SmallConfig()
eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

In [ ]:
data_len = len(train_data)

In [9]:
train_data = tf.convert_to_tensor(train_data, dtype=tf.int32, name="train_data")

In [22]:
batch_len = data_len // config.batch_size

In [23]:
print(batch_len)

46479


In [26]:
data = tf.reshape(train_data[0 : config.batch_size * batch_len],
                      [config.batch_size, batch_len])

In [27]:
data.shape

TensorShape([Dimension(20), Dimension(46479)])

In [28]:
epoch_size = (batch_len - 1) // config.num_steps

In [30]:
epoch_size = tf.identity(epoch_size, name="epoch_size")

In [31]:
epoch_size

<tf.Tensor 'epoch_size:0' shape=() dtype=int32>

In [41]:
i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()

In [64]:
x = tf.strided_slice(data, [0, i * config.num_steps],
                         [config.batch_size, (i + 1) * config.num_steps])
x.set_shape([config.batch_size, config.num_steps])
#在data词矩阵中获取参与第i次迭代的训练词对应的目标词
y = tf.strided_slice(data, [0, i * config.num_steps + 1],
                         [config.batch_size, (i + 1) * config.num_steps + 1])
y.set_shape([config.batch_size, config.num_steps])

(20, 20)


In [66]:
init = tf.global_variables_initializer()
sess = tf.Session()
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
sess.run(init)
try:

    output_x = sess.run(x)
    output_y = sess.run(y)
    print("x : ", output_x)
    print("y : ", output_y)
#     index = 0
#     while not coord.should_stop() and index<10:
#         datalines = sess.run(x)
#         index += 1
#         print("step: %d, batch data: %s" % (index, datalines.shape))
# except tf.errors.OutOfRangeError:
#     print("Done traing:-------Epoch limit reached")
# except KeyboardInterrupt:
#     print("keyboard interrput detected, stop training")
finally:
    coord.request_stop()
coord.join(threads)
sess.close()
del sess

x :  [[9970 9971 9972 9974 9975 9976 9980 9981 9982 9983 9984 9986 9987 9988
  9989 9991 9992 9993 9994 9995]
 [  10   42  105 3195    6 1353   52   22  207   10   14   42    1 1268
    36    6 1353    9 1139    5]
 [   3   71    4   27  246   60   11  215    4    1 1846    9    3   71
   546    2 6505  162    6  104]
 [   8  446  398    2   10  863  803   79   25 5175 2476    5  526 4787
  2151  270   28   15    2 1206]
 [  18  296    7  201   76    4  182  560 3836   17  974  975    6  942
     4  156   77 1570  288  644]
 [1028    0  332 7147  328 1452 8595   59 1569  105 2231    1  895    1
  5574    4  618    2   29   39]
 [   2  179  940  117   38   59  677   14    1   10 1016  309   13 1077
  6360   16   23 4490    9  355]
 [1261    0    1    0 2172 1261    0    1    2   23 2662    8   23 3049
  7386    7  769    1    7    0]
 [1473   88   19 2578 6591    8  629  563    8  223  184  127   18    6
   828    1    2    0  324  158]
 [ 155 4404    2  894 1585   96 5481 2148   15  91

In [ ]:
cell = tf.contrib.rnn.BasicLSTMCell()